In [ ]:
from create_table import create_table, delete_table
# delete_table()
create_table()

In [1]:
import pandas as pd
import datetime

file_path = 'dataset/diary_content.csv'
columns = ['timestamp', 'content']
diary = pd.read_csv(file_path, names=columns, header=None)

# 전처리
diary['timestamp'] = pd.to_datetime(diary['timestamp'])
diary['content'] = diary['content'].str.replace("’", "")

diary['year'] = diary['timestamp'].dt.year.astype('int32')
diary['month'] = diary['timestamp'].dt.month.astype('int32')
diary['day'] = diary['timestamp'].dt.day.astype('int32')
diary['hour'] = diary['timestamp'].dt.hour.astype('int32')
diary['minute'] = diary['timestamp'].dt.minute.astype('int32')
print(diary)

             timestamp                                            content  \
0  2023-08-01 14:20:00  Excitement fills the air as I embark on a new ...   
1  2023-08-02 10:45:00  Rainy morning vibes matched my introspective m...   
2  2023-08-03 17:12:00  Unexpected encounter with an old friend bright...   
3  2023-08-04 19:30:00  Completed a challenging project at work. Relie...   
4  2023-08-05 13:55:00  Nature walk and a picnic—today was a breath of...   
..                 ...                                                ...   
95 2023-11-04 11:20:00  Rainy afternoon spent writing letters to loved...   
96 2023-11-05 19:08:00  Hosted a board game night with friends. Strate...   
97 2023-11-06 22:30:00  Late-night poetry reading by candlelight. Vers...   
98 2023-11-07 16:15:00  Visited a botanical garden and marveled at the...   
99 2023-11-08 09:48:00  Morning bike ride through the city. Each turn ...   

    year  month  day  hour  minute  
0   2023      8    1    14      20  
1

In [2]:
# CSV 파일로 저장
selected_columns = ['year', 'month', 'day', 'hour', 'minute', 'content']
diary_selected = diary[selected_columns]
diary_selected.to_csv('dataset/diary_selected.csv', index=False)

In [6]:
# 100개의 diary를 DB에 저장
import psycopg2

connection_info = "host=147.47.200.145 dbname=teamdb8 user=team8 password=youngjoon port=34543"
conn = psycopg2.connect(connection_info)

try:
    cursor = conn.cursor()
    
    cursor.execute("""
        SELECT setval('date_id_seq', (SELECT COALESCE(MAX(id), 1) FROM date), false);
    """)

    # Insert data into the date table
    for i in range(len(diary)):
        year = int(diary.loc[i, 'year'])
        month = int(diary.loc[i, 'month'])
        day = int(diary.loc[i, 'day'])
        hour = int(diary.loc[i, 'hour'])
        minute = int(diary.loc[i, 'minute'])

        cursor.execute("""
            INSERT INTO date(year, month, day, hour, minute) VALUES (%s, %s, %s, %s, %s) RETURNING id
        """, (year, month, day, hour, minute))
        date_id = cursor.fetchone()[0]  # Retrieve the generated id

        # Insert data into the diary table
        content = str(diary.loc[i, 'content'])
        cursor.execute("""
            INSERT INTO diary(id, content) VALUES (%s, %s)
        """, (date_id, content))

    # Commit the changes
    conn.commit()

except psycopg2.Error as e:
    print("DB error: ", e)
    conn.rollback()

finally:
    conn.close()